<a href="https://colab.research.google.com/github/WasikulIslam1/CIS-4400-Project-9/blob/main/LivingwageETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install azure-storage-blob
!pip install pyarrow
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import datetime
import calendar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00


In [ ]:


# Read Azure Blob Storage connection string from config.json
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Get connection string from config
connection_string = config['connectionString']

# Create BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_name = 'livingwage'
blob_name = 'fbclivingwage'
download_file_path = 'fbclivingwage.csv'

blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

with open(download_file_path, 'wb') as download_file:
    download_file.write(blob_client.download_blob().readall())

print(f"Blob {blob_name} has been downloaded")

Blob fbclivingwage has been downloaded


In [ ]:
df = pd.read_csv('fbclivingwage.csv')
Data_df = df.copy()
pd.set_option('display.max_columns', None)

Data_df.head(5)

,Ranges for living wages: all counties in the US,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,EPI Family Budget Calculator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Two adults, two children",NaN,NaN,NaN,"Two adults, two children",NaN,NaN,NaN,"Two adults, two children",NaN,NaN,NaN
1,What constitutes a living wage?,NaN,NaN,Summary statistics (within county),NaN,NaN,"One adult, no children",NaN,NaN,"One adult, one child",NaN,NaN,NaN,One full-time worker,NaN,NaN,NaN,"Two workers, one full-time, one part-time",NaN,NaN,NaN,Two full-time workers,NaN,NaN,NaN
2,state_alpha,county_fips,county_name,Min,Median,Max,All income from wages,81% of income from wages (CBO est.),Employer provided healthcare,All income from wages,81% of income from wages (CBO est.),Employer provided healthcare,Child care capped at 7% of income,All income from wages,81% of income from wages (CBO est.),Employer provided healthcare,Child care capped at 7% of income,All income from wages,81% of income from wages (CBO est.),Employer provided healthcare,Child care capped at 7% of income,All income from wages,81% of income from wages (CBO est.),Employer provided healthcare,Child care capped at 7% of income
3,AL,1001,Autauga County,17.7720955236383,26.076836924592,45.9229134095577,21.9408586711584,17.7720955236383,19.5404548250045,32.1936258328296,26.076836924592,28.6005330646822,31.4463129177471,45.9229134095577,37.1975598617417,38.736727873263,43.3001663544901,30.6152756063718,24.7983732411611,25.824485248842,28.8667775696601,22.9614567047788,18.5987799308708,19.3683639366315,21.650083177245
4,AL,1003,Baldwin County,18.8079419517347,27.7694722630032,48.3125390826664,23.2196814218947,18.8079419517347,20.9913160372793,34.2832990901275,27.7694722630032,30.9477255527493,33.3945940102627,48.3125390826664,39.1331566569598,41.6413920079101,45.199536012618,32.2083593884442,26.0887711046398,27.7609280052734,30.133024008412,24.1562695413332,19.5665783284799,20.820696003955,22.599768006309


In [ ]:
new_column_names = Data_df.iloc[2].tolist()
Data_df.columns = new_column_names


In [ ]:
# Remove the first 3 rows
Data_df = Data_df.iloc[3:]

# Reset index if needed
Data_df.reset_index(drop=True, inplace=True)
Data_df.head()

,state_alpha,county_fips,county_name,Min,Median,Max,All income from wages,81% of income from wages (CBO est.),Employer provided healthcare,All income from wages,81% of income from wages (CBO est.),Employer provided healthcare,Child care capped at 7% of income,All income from wages,81% of income from wages (CBO est.),Employer provided healthcare,Child care capped at 7% of income,All income from wages,81% of income from wages (CBO est.),Employer provided healthcare,Child care capped at 7% of income,All income from wages,81% of income from wages (CBO est.),Employer provided healthcare,Child care capped at 7% of income
0,AL,1013,Butler County,16.172909586674,23.2960014301836,41.4691468067992,19.9665550452765,16.172909586674,17.639593506815,28.7604955928193,23.2960014301836,25.2773364198327,27.6232637026079,41.4691468067992,33.5900089135073,34.5028284608259,38.1050538876595,27.6460978711994,22.3933392756715,23.0018856405506,25.4033692584397,20.7345734033996,16.7950044567536,17.2514142304129,19.0525269438297
1,AL,1015,Calhoun County,15.7434402848891,22.7600544413564,40.9320915771812,19.4363460307273,15.7434402848891,17.1584229538042,28.0988326436499,22.7600544413564,24.6890776779583,27.2703622716056,40.9320915771812,33.1549941775168,34.112581645798,38.2065606733842,27.2880610514541,22.1033294516778,22.7417210971987,25.4710404489228,20.4660457885906,16.5774970887584,17.056290822899,19.1032803366921
2,AL,1017,Chambers County,16.2618124728332,23.917007041684,42.2399471640099,20.0763116948558,16.2618124728332,17.8479463102404,29.5271691872642,23.917007041684,26.1915956498861,28.1742655732945,42.2399471640099,34.214357202848,35.5688000892537,38.5682280035997,28.1599647760066,22.8095714685653,23.7125333928358,25.7121520023998,21.1199735820049,17.107178601424,17.7844000446268,19.2841140017998
3,AL,1019,Cherokee County,15.4200887483983,21.5777804115334,38.0742932059219,19.2389051446317,15.5835131671516,16.9119436061701,26.6392350759672,21.5777804115334,23.1560759029806,26.593183340821,38.0742932059219,30.8401774967967,31.1079748599486,36.9178973494085,25.3828621372812,20.5601183311978,20.7386499066324,24.6119315662723,19.0371466029609,15.4200887483983,15.5539874299743,18.4589486747042
4,AL,1021,Chilton County,16.3030951718896,23.3180650160623,42.0405596590536,20.1272779899872,16.3030951718896,17.6778933746026,28.7877345877312,23.3180650160623,25.1213239701797,27.7032615529195,42.0405596590536,34.0528533238334,34.7077384239506,38.9541697622907,28.0270397727024,22.7019022158889,23.1384922826337,25.9694465081938,21.0202798295268,17.0264266619167,17.3538692119753,19.4770848811453


In [ ]:

LW_df = Data_df[['county_fips', 'county_name', 'Min', 'Median', 'Max']].copy()
LW_df.head(24)


,county_fips,county_name,Min,Median,Max
0,1013,Butler County,16.172909586674,23.2960014301836,41.4691468067992
1,1015,Calhoun County,15.7434402848891,22.7600544413564,40.9320915771812
2,1017,Chambers County,16.2618124728332,23.917007041684,42.2399471640099
3,1019,Cherokee County,15.4200887483983,21.5777804115334,38.0742932059219
4,1021,Chilton County,16.3030951718896,23.3180650160623,42.0405596590536
5,1023,Choctaw County,15.6961237341372,22.4746330788268,40.0592608897721
6,1025,Clarke County,15.5866207335546,22.9961817191606,41.4605909247143
7,1027,Clay County,15.3173045184605,22.1389747796183,38.957441138626
8,1029,Cleburne County,15.6629119735539,22.3517115390942,39.7128191119991
9,1031,Coffee County,16.130383249963,22.9011677428876,41.0228986890876
